In [2]:
from libraries import *
from parameters import *
import statsmodels.api as sm
import sklearn.feature_selection
import statsmodels.api as sm
import sklearn.feature_selection
from sklearn.preprocessing import PolynomialFeatures
import warnings
warnings.filterwarnings('ignore')
#from statannot import add_stat_annotation
from numpy.random import seed
from numpy.random import randint
import statsmodels.formula.api as smf

import statsmodels.api as sm
%load_ext rpy2.ipython

import sys

In [ ]:
setIndex_1=0

In [ ]:
setIndex_2= setIndex_1+150
print(setIndex_2)

In [ ]:
os.getcwd()
os.chdir(projectDir)

In [ ]:
adata = sc.read('outputs/anndata/adata-hash-features_singlets_SingleKO_05242020_Filtered.h5ad')
allGeneNames = pd.read_csv("/home/beraslan/jovian-work/analysisSingle/AllExpGeneNames.csv", index_col=0)

In [ ]:
allBarcodeNames = adata.uns['feature_barcode_names']
controlBarcodes = allBarcodeNames[[(x.startswith("NO_TARGET_") or x.startswith("ONE_NONGENE_SITE_")) for x in allBarcodeNames]]
adata.obs["ControlGuidesAll"] = adata.obs[controlBarcodes].sum(axis=1)

In [ ]:
guideMatrix = adata.obs[adata.uns['feature_KO_barcode_names_filtered']]
KOGuides = pd.Series(adata.uns['feature_KO_barcode_names_filtered'])
guideMatrix.shape

In [ ]:
KOGuides

In [ ]:
df = adata.obs[["n_genes", "mt_frac", "leiden" ]]
# dummies = pd.get_dummies(df.leiden)
# df = df.join(dummies)
# df.drop(["leiden"], axis=1,inplace = True) 
# df.shape

In [ ]:
guideMatrix = guideMatrix.join(df)
guideMatrix.shape

In [ ]:
expressionMatrix = pd.DataFrame(adata.raw.X.A)
expressionMatrix.columns = allGeneNames.iloc[:,0]
expressionMatrix = expressionMatrix[adata.var_names]
expressionMatrix.index = adata.obs.index
expressionMatrix.shape

In [ ]:
selColumns = pd.Series(KOGuides[setIndex_1:setIndex_2]).append(pd.Series(["n_genes", "mt_frac", "leiden"]))

In [ ]:
selColumns

In [ ]:
controlExpressionMat = expressionMatrix.loc[adata.obs["ControlGuidesAll"] == 1,]
controlGuideMat = guideMatrix.loc[adata.obs["ControlGuidesAll"] == 1,]
controlExpressionMat = controlExpressionMat.iloc[0:5000,:]
controlGuideMat = controlGuideMat.iloc[0:5000,:]
controlGuideMat=controlGuideMat[selColumns]

In [ ]:
controlGuideMat

In [ ]:
geneExpressionMat = expressionMatrix.loc[adata.obs[KOGuides[setIndex_1:setIndex_2]].sum(axis=1) > 0,]
geneGuideMat = guideMatrix.loc[adata.obs[KOGuides[setIndex_1:setIndex_2]].sum(axis=1) > 0,]
geneGuideMat=geneGuideMat[selColumns]

In [ ]:
geneExpressionMat

In [ ]:
geneGuideMat

In [ ]:
allExpMat = np.concatenate((controlExpressionMat, geneExpressionMat))
allGuideMat = controlGuideMat.append(geneGuideMat)

In [ ]:
allVars = pd.Series(KOGuides[setIndex_1:setIndex_2]) 
myForm = allVars.append(pd.Series(["n_genes","mt_frac","leiden"]))

myFormula = "+".join(myForm)
my_formula = "y~" + myFormula

In [ ]:
my_formula

In [ ]:
allResp=adata.var_names
allResp

In [ ]:
del adata 

In [ ]:
%%R -i allGuideMat,allExpMat,my_formula,allResp,setIndex_1,setIndex_2  -w 8 -h 4 -u in
library("MASS")
library("broom")

for(i in seq(1,ncol(allExpMat),20)){
    print(i)
    
    coefDF = data.frame()
    for(j in seq(i,i+20,1)){
       
        
      tryCatch(
            expr = {
                   allGuideMat["y"] = allExpMat[,j]
                   myFit <- glm.nb(formula(my_formula), data = allGuideMat)
                   myDF <- data.frame(tidy(myFit))
                   myDF$respGene = allResp[j] 
                   coefDF <- rbind(coefDF, myDF)
            },
            error = function(e){ 
                print("alooooo")
                print(e)
            },
            finally = {
                # (Optional)
                # Do this at the end before quitting the tryCatch structure...
            }
      )
     
    }
    
     print(dim(coefDF))
     saveRDS(coefDF, paste0("./GuidesLM_NegativeBinomial/coefs_",setIndex_1,"_",setIndex_2,"_", i, "_",i+20,".rds"))
    

}

In [ ]:
%%R  -w 8 -h 4 -u in

readRDS("./GuidesLM_NegativeBinomial/)